In [1]:
import pandas as pd
import os
import ast
import time
import csv
import datetime

In [2]:
def create(country,filename):
    print('Creating....')
    os.system('alpr -c {} -j {} C:\OpenAlpr\openalpr_64\openalpr.conf\
    > test2.txt'.format(country,filename))
    detect()

In [3]:
def detect():
    print('Detecting....')
    data = []
    dic = {}
    file = open('test2.txt','r')
    contents = file.readlines()
    idx = contents[0].index("candidates")
    clean = contents[0][idx+13:][:-5]
    data = clean.split(',"matches_template":0}')
    cleaned_data = data[0][1:]
    dic = ast.literal_eval('{'+cleaned_data+'}')
    plate = dic['plate']
    output(plate)

In [4]:
def output(plate):
    print('Processing....')
    df = pd.read_csv('dbase.csv')
    z = 0
    for i in df['Number Plate']:
        z+=1
        if i == plate:
            indx = z
    print('Result - \n')
    results = df.iloc[[indx-1]].reset_index()
    del results['index']
    print(df.iloc[indx-1])
    invoice(results)

In [5]:
def invoice(dframe):
    date = time.strftime('%Y-%m-%d %H:%M:%S')
    modified_date = datetime.datetime.now() + datetime.timedelta(days = 30)
    modified_date = datetime.datetime.strftime(modified_date, "%Y-%m-%d %H:%M:%S")
    fdat = pd.DataFrame({'Date': [date]})
    tdat = pd.DataFrame({'last Date': [modified_date]})
    new_df = pd.concat([dframe,fdat,tdat],axis = 1)
    with open('results.csv','a') as fd:
        new_df.to_csv(fd, header=False,encoding="utf-8", index=False)
    #with open("results.csv", "r") as f:
        #contents = f.read().encode("UTF-8")
        #contents.to_csv(f.decode())
    os.system('index.html')

In [6]:
def begin():
    print('Enter Country Code')
    ccode = input()
    print('Enter Image name')
    iname = input()+'.jpg'
    create(ccode,iname)

In [12]:
begin()

Enter Country Code
us
Enter Image name
us-4
Creating....
Detecting....
Processing....
Result - 

Number Plate                  LTM378
Name               Virginia Bradshaw
Phone Number            +41773589245
Address         5325 Virginia Street
Country                United States
Name: 10, dtype: object
